In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

from datetime import datetime, timedelta

In [323]:
### indicate the file paths
files = {
    'f1': ['20240408_Trials\htx3/2024-04-08\S057_20240408_rawDataLeft_zWl9F52Ehw.txt', '20240408_Trials\htx3/2024-04-08\S057_20240408_rawDataRight_zWl9F52Ehw.txt'],	
    'f2': ['20240408_Trials\htx3/2024-04-08\S058_20240408_rawDataLeft_zWl9F52Ehw.txt', '20240408_Trials\htx3/2024-04-08\S058_20240408_rawDataRight_zWl9F52Ehw.txt'],
    'f3': ['20240408_Trials\htx3/2024-04-08\S050_20240408_rawDataLeft_zWl9F52Ehw.txt', '20240408_Trials\htx3/2024-04-08\S050_20240408_rawDataRight_zWl9F52Ehw.txt'],
    'f4': ['20240408_Trials\htx3/2024-04-08\S056_20240408_rawDataLeft_zWl9F52Ehw.txt', '20240408_Trials\htx3/2024-04-08\S056_20240408_rawDataRight_zWl9F52Ehw.txt'],
    'f5': ['20240408_Trials\htx3/2024-04-08\S059_20240408_rawDataLeft_zWl9F52Ehw.txt', '20240408_Trials\htx3/2024-04-08\S059_20240408_rawDataRight_zWl9F52Ehw.txt'],
    }

### indicate date of the activity
date = datetime(2024, 4, 8)

### participants
participants = {
    'id': ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12'],
    'name': ['Syawal', 'Salihin', 'Luther', 'Ilyas', 'Bryan', 'Benyamin', 'Ridwan', 'Asrudin', 'STAR', 'Afiq', 'Sean', 'Shawn'],
    'weight': [77.4, 60.9, 69.2, 67.85, 63.8, 68.9,],
    'shoesize': [9, 8, 10, 9, 9, 9.5, 8, 7.5, 10.5, 9.5, 10.5, 9.5]
}


In [69]:
# load files
def load_file(path):
    df = pd.read_csv(path, sep=',', header=None)
    if df.shape[1] == 26:
        df = df.iloc[:, :-2]
    df.columns = [
        'index_no', 'raw_1', 'raw_2', 'raw_3', 'raw_4', 'raw_5', 'raw_6', 'raw_7',
        'raw_8', 'raw_9', 'raw_10', 'raw_11', 'raw_12', 'raw_13','raw_14',
        'acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'sess_no', 'Timestamp', 'distance_ph'
    ]
    df.drop(columns=['index_no', 'distance_ph'], inplace=True)
    return df

In [280]:
def resample(df):


    # df = df[df['Timestamp'] >= 85200] # if required to filter data
    # df = df[df['Timestamp'] <= 102300]


    ms = []
    timestamps = df['Timestamp'].unique()
    # create timestamp for each reading within a second
    for second in timestamps:
        data = df[df['Timestamp'] == second]
        instances = len(data)
        ms_interval = 1000 // instances         # 1000 milliseconds in one second
        for i in range(instances):
            instance = datetime.combine(date.date(), (datetime.strptime(str(second), '%H%M%S') + timedelta(milliseconds=ms_interval*i)).time())
            ms.append(instance)
    # resample timeseries to 10ms
    df['ms'] = ms
    df = df.set_index('ms')
    df = df.resample('10ms').ffill()
    df = df.reset_index()
    return df.drop(columns=['Timestamp'])

In [106]:
def merge_lr(left, right):
    df = pd.merge(left, right, on='ms', how='inner', suffixes=('_left', '_right'))
    df['session'] = df['sess_no_left']
    df.set_index('session', inplace=True)
    df.reset_index(inplace=True)
    return df.drop(columns=['sess_no_left', 'sess_no_right'])

In [324]:
left = resample(load_file(files['f1'][0]))
right = resample(load_file(files['f1'][1]))

In [325]:
df = merge_lr(left, right)
df

,session,ms,raw_1_left,raw_2_left,raw_3_left,raw_4_left,raw_5_left,raw_6_left,raw_7_left,raw_8_left,...,raw_11_right,raw_12_right,raw_13_right,raw_14_right,acc_x_right,acc_y_right,acc_z_right,gyro_x_right,gyro_y_right,gyro_z_right
0,57,2024-04-08 10:31:11.000,18,28,22,13,12,26,3,19,...,15,29,30,23,-8267,1009,-298,-1238,-95,-1590
1,57,2024-04-08 10:31:11.010,18,28,22,13,12,26,3,19,...,15,29,30,23,-8267,1009,-298,-1238,-95,-1590
2,57,2024-04-08 10:31:11.020,18,28,22,13,12,26,3,19,...,15,29,30,23,-8267,1009,-298,-1238,-95,-1590
3,57,2024-04-08 10:31:11.030,18,28,22,13,12,26,3,19,...,15,29,30,23,-8267,1009,-298,-1238,-95,-1590
4,57,2024-04-08 10:31:11.040,17,26,23,14,12,26,2,19,...,15,29,30,23,-8267,1009,-298,-1238,-95,-1590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175190,57,2024-04-08 11:00:22.900,15,29,26,16,22,30,23,9,...,7,21,23,8,-2156,3420,-730,-51,-383,-4023
175191,57,2024-04-08 11:00:22.910,16,26,26,15,21,31,24,11,...,7,21,23,8,-2156,3420,-730,-51,-383,-4023
175192,57,2024-04-08 11:00:22.920,16,26,26,15,21,31,24,11,...,7,21,23,8,-2156,3420,-730,-51,-383,-4023
175193,57,2024-04-08 11:00:22.930,17,27,25,14,23,32,24,12,...,7,21,23,8,-2156,3420,-730,-51,-383,-4023


In [326]:
### load other dataframes if applicable

df1 = merge_lr(resample(load_file(files['f2'][0])), resample(load_file(files['f2'][1])))
df2 = merge_lr(resample(load_file(files['f3'][0])), resample(load_file(files['f3'][1])))
df3 = merge_lr(resample(load_file(files['f4'][0])), resample(load_file(files['f4'][1])))
df4 = merge_lr(resample(load_file(files['f5'][0])), resample(load_file(files['f5'][1])))
df1

,session,ms,raw_1_left,raw_2_left,raw_3_left,raw_4_left,raw_5_left,raw_6_left,raw_7_left,raw_8_left,...,raw_11_right,raw_12_right,raw_13_right,raw_14_right,acc_x_right,acc_y_right,acc_z_right,gyro_x_right,gyro_y_right,gyro_z_right
0,58,2024-04-08 11:00:48.000,13,28,24,13,13,30,8,10,...,22,32,32,24,-4250,928,1248,-363,-546,-417
1,58,2024-04-08 11:00:48.010,13,28,24,13,13,30,8,10,...,22,32,32,24,-4250,928,1248,-363,-546,-417
2,58,2024-04-08 11:00:48.020,13,28,24,13,13,30,8,10,...,22,32,32,24,-4250,928,1248,-363,-546,-417
3,58,2024-04-08 11:00:48.030,13,28,24,13,13,30,8,10,...,22,32,32,24,-4250,928,1248,-363,-546,-417
4,58,2024-04-08 11:00:48.040,13,28,24,13,13,30,8,10,...,21,32,31,23,-4127,1281,1394,-203,-486,-305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92387,58,2024-04-08 11:16:11.870,3,29,24,13,12,22,5,28,...,25,32,32,21,-3833,1402,1060,108,145,-128
92388,58,2024-04-08 11:16:11.880,3,29,24,13,12,22,5,28,...,25,32,32,21,-3833,1402,1060,108,145,-128
92389,58,2024-04-08 11:16:11.890,3,29,24,13,12,22,5,28,...,25,32,32,21,-3833,1402,1060,108,145,-128
92390,58,2024-04-08 11:16:11.900,3,27,23,13,12,21,6,25,...,25,32,32,21,-3833,1402,1060,108,145,-128


In [327]:
### join dataframes if applicable
df = pd.concat([df, df1], ignore_index=True)

In [333]:
# refer to participants list for htxid to participant id
df.to_csv('S0{}_20240408_rawData_{}.csv'.format(df['session'][0], participants['id'][6]), index=False)